In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

2023-12-04 22:14:17.494956: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-04 22:14:17.495056: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-04 22:14:17.519570: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-04 22:14:17.579026: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-04 22:14:18.702609: W tensorflow/compiler/tf2

In [69]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0)
plt.rcParams['image.cmap'] = 'Greys'

print(tf.__version__)
print(tf.config.get_visible_devices())
print("Num GPUs Available: ", tf.config.list_physical_devices('GPU'))

2.15.0
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Num GPUs Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [70]:
(ds_train, ds_test) = tfds.load('mnist', split=['train', 'test'], shuffle_files=True, as_supervised=True)
print(ds_train)

<_PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>


In [71]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(normalize_img)
ds_train = ds_train.batch(100)

ds_test_norm = ds_test.map(normalize_img)
ds_test = ds_test_norm.batch(100)

(ds_train, ds_test)

(<_BatchDataset element_spec=(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <_BatchDataset element_spec=(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>)

In [72]:
# wx + b
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Softmax()
])

# loss fn
cross_en = tf.keras.losses.SparseCategoricalCrossentropy(reduction=tf.keras.losses.Reduction.SUM)

model.compile(loss=cross_en, metrics=['accuracy'])

In [73]:
checkpoint = "training-simple-mnist/1000.ckpt"

try:
    model.load_weights(checkpoint)
except Exception as e:
    print(e)
    model.fit(ds_train, epochs=1000)
    model.save_weights(checkpoint)

print(model.summary())
print(model.get_config())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                7850      
                                                                 
 softmax_2 (Softmax)         (None, 10)                0         
                                                                 
Total params: 7850 (30.66 KB)
Trainable params: 7850 (30.66 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
{'name': 'sequential_2', 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_input_shape': (None, 28, 28), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'flatten_2_input'}, 'registered_name': None}, {'module': 'keras.layers', 'clas

In [81]:
# overall test
test_loss, test_acc = model.evaluate(ds_test.take(1), verbose=2)
print('\nTest accuracy:', test_acc)

1/1 - 0s - loss: 25.3632 - accuracy: 0.9200 - 42ms/epoch - 42ms/step

Test accuracy: 0.9200000166893005


2023-12-04 23:58:00.083828: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [88]:
# wx + b + relu
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(100, activation='relu'),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Softmax()
])

# loss fn
cross_en = tf.keras.losses.SparseCategoricalCrossentropy(reduction=tf.keras.losses.Reduction.SUM)

model.compile(loss=cross_en, metrics=['accuracy'])

print(model.summary())
print(model.get_config())

checkpoint = "training-simple-mnist/100_relu.ckpt"

try:
    model.load_weights(checkpoint)
except Exception as e:
    print(e)
    model.fit(ds_train, epochs=100)
    model.save_weights(checkpoint)



# overall test
test_loss, test_acc = model.evaluate(ds_test.take(1), verbose=2)
print('\nTest accuracy:', test_acc)

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_9 (Flatten)         (None, 784)               0         
                                                                 
 dense_12 (Dense)            (None, 100)               78500     
                                                                 
 dense_13 (Dense)            (None, 10)                1010      
                                                                 
 softmax_8 (Softmax)         (None, 10)                0         
                                                                 
Total params: 79510 (310.59 KB)
Trainable params: 79510 (310.59 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
{'name': 'sequential_8', 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_input_shape': (None, 28, 28), 'dty

2023-12-05 00:55:59.831903: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


1/1 - 0s - loss: 10.0891 - accuracy: 0.9800 - 131ms/epoch - 131ms/step

Test accuracy: 0.9800000190734863
